<a href="https://colab.research.google.com/github/bangkit-cap0423/Guifena-ML/blob/master/Bangkit_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install ffmpeg


In [ ]:
pip install scaper

In [ ]:
import scaper, os
os.getcwd()

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
os.chdir('drive/MyDrive/audio_alt_path')

# Download from youtube dynamically

In [ ]:

pip install youtube_dl
import youtube_dl


ydl_opts = {
    'format': 'bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
        'preferredquality': '192',
    }],
}
linkchainsaw = ['https://www.youtube.com/watch?v=0DzsPL-xElE','https://www.youtube.com/watch?v=1MgcrYdYas0',
                'https://www.youtube.com/watch?v=9HcahqYUVoc','https://www.youtube.com/watch?v=AhcY8QVSLtM',
                'https://www.youtube.com/watch?v=BBukw6JpCeg','https://www.youtube.com/watch?v=BsYFCAuPjwE', 
                'https://www.youtube.com/watch?v=C46X66FU_Dw', 'https://www.youtube.com/watch?v=LH08k5Kf4AI', 
                'https://www.youtube.com/watch?v=NJUl3gPX07o', 'https://www.youtube.com/watch?v=_6uZ1HyHSQY',
                'https://www.youtube.com/watch?v=kHNWRR0hJ08','https://www.youtube.com/watch?v=yaaHJV5VOXQ']

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
  for link in linkchainsaw:
    ydl.download([link])

In [ ]:
os.chdir('/content/drive/MyDrive/audio_alt_path')

# Generate Dataset Chainsaw sound + Forest Sound

In [ ]:
import scaper, os
soundscape_duration = 30.0
outfolder = 'soundscapes/'
foreground = os.path.expanduser("foreground")
background = os.path.expanduser("background")
sc = scaper.Scaper(soundscape_duration, foreground, background)
sc.add_background(label=("const", "forest"),
                 source_file=("choose", []),
                 source_time=("uniform", 0, 300-soundscape_duration))

for i in range(1000):
    audiofile = outfolder+f'/normal_{i}.wav'
    jamsfile = outfolder+f'/normal_{i}.jams'
    txtfile = outfolder+f'/normal_{i}.txt'
    sc.generate(audiofile, jamsfile,
                allow_repeated_label=True,
                allow_repeated_source=True,
                disable_sox_warnings=True,
                no_audio=False)
    
sc.add_event(label=("const", "chainsaw"),
            source_file=("choose", []),
            source_time=("uniform", 10, 30),
            event_time=("const", 0),
            event_duration=("const", 30),
            snr=("uniform", -5, 0),
            pitch_shift=("uniform", -15, 15),
            time_stretch=None)

for i in range(1000):
    audiofile = outfolder+f'/soundscape_{i}.wav'
    jamsfile = outfolder+f'/soundscape_{i}.jams'
    sc.generate(audiofile, jamsfile,
                allow_repeated_label=True,
                allow_repeated_source=True,
                disable_sox_warnings=True,
                no_audio=False)

# Generate Image Dataset (1) Due too Memory Usage, Split into 2





In [ ]:
import librosa.display, os, gc
import numpy as np
import matplotlib.pyplot as plt

def extract_spectrogram(fname, iname):
    audio, sr = librosa.load(fname, res_type='kaiser_fast')
    S = librosa.feature.melspectrogram(audio, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    fig = plt.figure(figsize=[1, 1])
    ax = fig.add_subplot(111)
    fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
    ax.axis("off")
    ax.axis("tight")
    plt.margins(0)
    librosa.display.specshow(log_S, sr=sr)
    fig.savefig(iname, dpi=100, pad_inches=0)
    plt.close(fig)
    plt.close('all')
    del audio, S, log_S, ax, fig
    
samples_folder = "soundscapes/"
images_folder = "images/"
already = os.listdir(images_folder)
d = os.listdir(samples_folder)
for i, f in enumerate(d):
    if(f.split('.')[1]=='wav'):
      extract_spectrogram(samples_folder+f, f"{images_folder}/{f.replace('.wav', '.png')}")

# Generate Image Dataset 2 Due too Memory Usage, Split into 2

In [ ]:
import librosa.display, os, gc
import numpy as np
import matplotlib.pyplot as plt

def extract_spectrogram(fname, iname):
    audio, sr = librosa.load(fname, res_type='kaiser_fast')
    S = librosa.feature.melspectrogram(audio, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    fig = plt.figure(figsize=[1, 1])
    ax = fig.add_subplot(111)
    fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
    ax.axis("off")
    ax.axis("tight")
    plt.margins(0)
    librosa.display.specshow(log_S, sr=sr)
    fig.savefig(iname, dpi=100, pad_inches=0)
    plt.close(fig)
    plt.close('all')
    del audio, S, log_S, ax, fig
    
samples_folder = "soundscapes/"
images_folder = "images/"
already = os.listdir(images_folder)
d = os.listdir(samples_folder)
for i, f in enumerate(d):
  name = f.split('.')[0]
  tipe,nomor = name.split('_')
  print(name)
  if(f.split('.')[1]=='wav'):
    if(tipe=='soundscape' and int(nomor)>300):
      extract_spectrogram(samples_folder+f, f"{images_folder}/{f.replace('.wav', '.png')}")

# Labeling and Split Data 

In [ ]:
 
import os
import tensorflow as tf
import numpy as np
samples = []
labels = []
images_folder = "images/"
for image in os.listdir(images_folder):
    samples.append(tf.keras.preprocessing.image.img_to_array(tf.keras.preprocessing.image.load_img(images_folder+image, target_size=(100, 100))))
    if "normal" in image:
        labels.append((0))
    else:
        labels.append((1))
samples = np.array(samples)
labels = np.array(labels)
eval_x, eval_y = samples[-100:], labels[-100:]
print(samples.shape, labels.shape)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(samples[:-100], labels[:-100], test_size=0.2, random_state=123)

# Training

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(100, 100, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
tf.keras.layers.Dense(512, activation='relu'),
model.add(Dense(1, activation='sigmoid'))
print(model.summary())
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(x_train, y_train, batch_size=20, epochs=150,
          validation_data=(x_test, y_test),
         verbose=1)
model.save('model/')


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and Validation Accuracy')

plt.figure()

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and Validation Loss')

print('Training Accuracy \t: {}'.format(acc[-1]))
print('Validation Accuracy \t: {}'.format(val_acc[-1]))
print('Training Loss \t\t: {}'.format(loss[-1]))
print('Validation Loss \t: {}'.format(val_loss[-1]))